In [14]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import flask

In [2]:
acct_rcv = pd.read_csv(r'C:\Users\Megan Finley\Documents\Lauren_Project\Data\accounts_receivable.csv',header=0)
claims = pd.read_csv(r'C:\Users\Megan Finley\Documents\Lauren_Project\Data\live_claims_feed_to7_30_2020.csv',header=0)

In [3]:
claims['bool'] = [1]*len(claims)

In [4]:
patients_ytd = [len(claims)]

In [5]:
patients_ytd

[1490]

In [6]:
claims['month'] = [int(claims["Date of Service"][j][0]) for j in range(len(claims))]
claims['quarter'] = [0]*len(claims)
for i in range(len(claims)):
    if claims['month'][i] <= 3:
        claims['quarter'][i] = 1
    elif claims['month'][i] > 3 and claims['month'][i] <= 6:
        claims['quarter'][i] = 2
    else:
        claims['quarter'][i] = 3


C:\Users\Megan Finley\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Megan Finley\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Megan Finley\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
months = [i for i in range(1,8)]
patients_monthly = []    
month_data = [claims[claims['month']== j] for j in range(1,8)]
quarter_data = [claims[claims['quarter']==j] for j in range(1,4)]
quarters = [i for i in range(1,4)]
patients_quarterly = []
for i in range(len(months)):
    patients_monthly.append(len(month_data[i]))
    
for i in range(len(quarters)):
    patients_quarterly.append(len(quarter_data[i]))
    

In [8]:
patients_monthly

[170, 191, 276, 243, 201, 130, 279]

In [9]:
patients_quarterly

[637, 574, 279]

In [10]:
patient_data = {'Month': patients_monthly,'Quarter': patients_quarterly, 'Year': patients_ytd}

In [11]:
drop_ops = ['Monthly','Quarterly','Yearly']

In [15]:

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

server = flask.Flask(__name__)
app = dash.Dash(__name__, server=server)

fig = go.Figure([go.Bar(x=months, y=patients_monthly)])

app.layout = html.Div([
    dcc.Graph(id='dropdown_graph'),
    dcc.Dropdown(
        id='x_axis',
        options=[
            {'label': 'Monthly', 'value': 'Month'},
            {'label': 'Quarterly', 'value': 'Quarter'},
            {'label': 'Year to Date', 'value': 'Year'}
        ],
        value= 'Month',
        style = dict(width = '80%',
                    display = 'inline-block',
                    verticalAlign = "middle")
    )
])


@app.callback(
    Output('dropdown_graph', 'figure'),
    [Input('x_axis', 'value')])
def update_figure(selected_option):
    data = patient_data[selected_option]
    if selected_option == 'Month':
        x = ['January', 'February', 'March','April', 'May', 'June', 'July']
    elif selected_option == 'Quarter':
        x = ['Q1', 'Q2','Q3']
    elif selected_option == 'Year':
        x = ['2020']
    
    fig = go.Figure([go.Bar(x=x, y=data)])
    
    fig.update_layout(title = 'Number of Patients Seen by '+selected_option, autosize = False, width = 1000, 
                      height = 750)
    
    fig.update_xaxes(title=selected_option) 

    fig.update_yaxes(title='Number of Patients') 
    

    return fig


if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 585-206-725
Debugger PIN: 585-206-725
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
